In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm
from sklearn.model_selection import train_test_split

In [2]:
#匯入user.movie互動資料
rating = pd.read_csv("Movielens\\user_movie.dat",sep = "\t",names=["userId","movieId","rating","timestamp"])
print("互動資料:")
rating

互動資料:


,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [3]:
#匯入電影+類型
movie = pd.read_csv("Movielens\\movie_genre.dat",sep = "\t",names=["movieId","genre"])
print("電影資料:")
movie

電影資料:


,movieId,genre
0,1,3
1,1,4
2,1,5
3,2,1
4,2,2
...,...,...
2886,1679,16
2887,1680,8
2888,1680,14
2889,1681,5


In [4]:
total = pd.merge(rating[["userId","movieId","rating"]], movie[["movieId","genre"]], on='movieId')
print("user+movie資訊合併:")
total

user+movie資訊合併:


,userId,movieId,rating,genre
0,196,242,3,5
1,63,242,3,5
2,226,242,5,5
3,154,242,3,5
4,306,242,5,5
...,...,...,...,...
212580,840,1674,4,8
212581,655,1640,3,8
212582,655,1637,3,8
212583,655,1630,3,8


In [5]:
#分割資料集
train_data, test_data = train_test_split(total, random_state=None, train_size=0.8)
print("Train data:")
train_data

Train data:


,userId,movieId,rating,genre
51389,452,419,4,4
15096,587,304,4,4
66386,648,483,5,14
159573,393,380,2,1
209146,393,541,3,2
...,...,...,...,...
139480,328,620,3,8
56290,328,177,3,1
28902,457,248,4,6
99156,593,66,5,5


In [6]:
print("Test data:")
test_data

Test data:


,userId,movieId,rating,genre
9187,254,1,3,5
139457,846,492,3,8
174179,13,83,2,5
165363,339,435,4,5
129386,227,121,2,17
...,...,...,...,...
106005,627,631,3,8
20523,178,181,5,15
116007,493,431,5,1
19161,118,100,5,16


In [7]:
#item-based

In [9]:
data_table_train = pd.pivot_table(train_data,values='rating',columns='movieId',index='userId')
data_table_train.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,1670,1671,1672,1673,1674,1675,1678,1679,1680,1681
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
data_table_test = pd.pivot_table(test_data,values='rating',columns='movieId',index='userId')
data_table_test.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,1656,1657,1659,1662,1666,1670,1673,1676,1677,1682
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,3.0,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
print("here are a list of 50 movies to recommend to a user who has liked 'movieId=1'(pred)")
print(data_table_train.corr()[1].sort_values(ascending=False).iloc[:50])
# print(data_table_train.corr()[1])
print("here are a list of 50 movies to recommend to a user who has liked 'movieId=1'(act)")
print(data_table_test.corr()[1].sort_values(ascending=False).iloc[:50])
# print(data_table_test.corr()[1])

here are a list of 50 movies to recommend to a user who has liked 'movieId=1'(pred)
movieId
1355    1.000000
1       1.000000
1255    1.000000
1398    1.000000
1392    1.000000
1389    1.000000
1385    1.000000
1372    1.000000
1365    1.000000
1351    1.000000
1323    1.000000
1287    1.000000
1275    1.000000
1242    1.000000
1442    1.000000
1233    1.000000
75      1.000000
1193    1.000000
1154    1.000000
1106    1.000000
1083    1.000000
1026    1.000000
920     1.000000
909     1.000000
757     1.000000
341     1.000000
1420    1.000000
296     1.000000
1501    1.000000
1450    1.000000
1445    1.000000
1544    1.000000
1484    1.000000
1465    1.000000
1464    1.000000
1629    1.000000
1479    1.000000
1598    1.000000
1658    1.000000
1600    1.000000
1560    1.000000
1497    1.000000
885     1.000000
1605    0.995871
1402    0.981981
1251    0.973329
884     0.970725
1344    0.970725
1529    0.944911
1438    0.944911
Name: 1, dtype: float64
here are a list of 50 movies to re

In [12]:
A=data_table_train.corr()[1]
B=data_table_test.corr()[1]
print(type(A))
print(B)
A

<class 'pandas.core.series.Series'>
movieId
1       1.000000
2       0.244587
3       0.455233
4       0.511427
5       0.795893
          ...   
1670         NaN
1673         NaN
1676         NaN
1677         NaN
1682         NaN
Name: 1, Length: 1449, dtype: float64


movieId
1       1.000000
2       0.225415
3       0.101511
4       0.103135
5       0.387265
          ...   
1675         NaN
1678         NaN
1679         NaN
1680         NaN
1681         NaN
Name: 1, Length: 1656, dtype: float64

In [13]:
Final_ratings = pd.merge(A,B,on='movieId')
Final_ratings

,1_x,1_y
movieId,,
1,1.000000,1.000000
2,0.225415,0.244587
3,0.101511,0.455233
4,0.103135,0.511427
5,0.387265,0.795893
...,...,...
1656,NaN,NaN
1657,NaN,NaN
1662,NaN,NaN


In [14]:
F=Final_ratings.dropna(axis=0, how='any')
print(F)

              1_x       1_y
movieId                    
1        1.000000  1.000000
2        0.225415  0.244587
3        0.101511  0.455233
4        0.103135  0.511427
5        0.387265  0.795893
...           ...       ...
1446     0.038180 -1.000000
1469     0.319122  0.500000
1478     0.042830  1.000000
1480    -0.213201 -1.000000
1615     0.672842  1.000000

[829 rows x 2 columns]


In [68]:
F.sort_values(by='1_x').tail(30)

,1_x,1_y
movieId,,
1300,0.534522,1.000000e+00
730,0.541215,3.333333e-01
928,0.541538,2.758386e-01
653,0.543120,-7.071068e-01
892,0.558389,4.417261e-01
1298,0.559893,1.000000e+00
853,0.565367,1.000000e+00
795,0.566706,4.909903e-01
1121,0.568290,1.000000e+00


In [16]:
type(F)

pandas.core.frame.DataFrame

In [19]:
# sort the dataframe
Final_ratings.sort_values(by='1_x',ascending=False,inplace=True)
# remove rows with missing values
Final_ratings.dropna(inplace=True)
# convert ratings to binary labels
threshold = 0.8
Final_ratings = Final_ratings >= threshold
# view results
Final_ratings.head(5)

,1_x,1_y
movieId,,
1058,True,False
639,True,False
1,True,True
650,True,False
935,True,False


In [16]:
def recall_at_k(df: pd.DataFrame, k, y_test: str='1_y', y_pred: str='1_x') -> float:
    """
    Function to compute recall@k for an input boolean dataframe
    
    Inputs:
        df     -> pandas dataframe containing boolean columns y_test & y_pred
        k      -> integer number of items to consider
        y_test -> string name of column containing actual user input
        y-pred -> string name of column containing recommendation output
        
    Output:
        Floating-point number of recall value for k items
    """
    # check we have a valid entry for k
    if k <= 0:
        raise ValueError('Value of k should be greater than 1, read in as: {}'.format(k))
    # check y_test & y_pred columns are in df
    if y_test not in df.columns:
        raise ValueError('Input dataframe does not have a column named: {}'.format(y_test))
    if y_pred not in df.columns:
        raise ValueError('Input dataframe does not have a column named: {}'.format(y_pred))
        
    # extract the k rows
    dfK = df.head(k)
    # compute number of all relevant items
    denominator = df[y_test].sum()
    # compute number of recommended items that are relevant @k
    numerator = dfK[dfK[y_pred] & dfK[y_test]].shape[0]
    # return result
    if denominator > 0:
        return numerator/denominator
    else:
        return None

In [24]:
k = 10
print('Recall@k: {:.2f} for k={}'.format(recall_at_k(Final_ratings,k),k))

Recall@k: 0.20 for k=10


In [19]:
from sklearn.metrics import ndcg_score

In [69]:
A=[[1.0,1.0,1.0,0.9428,0.8660,0.7536,0.7484,0.7364,0.6728,0.6545,0.6536,0.6437,0.6358,0.6182,0.6166,0.5991,0.5932,0.5773,0.5773,0.5773,0.5725,0.5682,0.5667,0.5653,0.5598,0.5583,0.5431,0.5415,0.5412,0.5434]]
B=[[1.0,1.0,1.0,1.0,1.0,4.6727,1.0,1.1102,5.0,7.5592,6.9663,-5.0,5.0,4.15,8.66,-1.0,5.7735,1.0,5.0,8.4515,9.1654,1.0,4.9099,1.0,1.0,4.4172,-7.0710,2.7583,3.3333,1.0]]

In [70]:
ndcg_score(A,B, k=10)

0.6868875334758757

In [21]:
#user-based

In [8]:
data_table_train = pd.pivot_table(train_data,values='rating',columns='userId',index='movieId')
data_table_train.head()

userId,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
movieId,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,2.0,3.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data_table_test = pd.pivot_table(test_data,values='rating',columns='userId',index='movieId')
data_table_test.head()

userId,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
movieId,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
2,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
print("here are a list of 50 movies to recommend to a user who has liked 'userId=1'(pred)")
print(data_table_train.corr()[1].sort_values(ascending=False).iloc[:50])
# print(data_table_train.corr()[1])
print("here are a list of 50 movies to recommend to a user who has liked 'userId=1'(act)")
print(data_table_test.corr()[1].sort_values(ascending=False).iloc[:50])
# print(data_table_test.corr()[1])

here are a list of 50 movies to recommend to a user who has liked 'userId=1'(pred)
userId
1      1.000000
166    1.000000
611    1.000000
531    1.000000
511    1.000000
656    1.000000
812    1.000000
485    1.000000
351    1.000000
171    1.000000
240    1.000000
873    1.000000
616    0.980196
520    0.968496
105    0.944911
107    0.935414
34     0.891042
926    0.884652
740    0.870388
179    0.866025
687    0.866025
400    0.866025
319    0.866025
510    0.862582
626    0.854238
803    0.849655
791    0.845489
304    0.845154
572    0.842927
266    0.837240
691    0.833119
696    0.816497
920    0.816497
61     0.814092
754    0.810643
17     0.810608
898    0.803116
4      0.796970
702    0.794948
628    0.774597
134    0.771744
736    0.766032
414    0.762713
550    0.750508
282    0.750000
100    0.748129
139    0.738549
93     0.733333
544    0.727607
596    0.720577
Name: 1, dtype: float64
here are a list of 50 movies to recommend to a user who has liked 'userId=1'(act)
user

In [11]:
C=data_table_train.corr()[1]
D=data_table_test.corr()[1]
D

userId
1      1.000000
2      0.870388
3           NaN
4      1.000000
5      0.298279
         ...   
939    0.294174
940   -0.113961
941    0.500000
942   -0.866025
943    0.456595
Name: 1, Length: 943, dtype: float64

In [12]:
Final_ratings = pd.merge(D,C,on='userId')
Final_ratings

,1_x,1_y
userId,,
1,1.000000,1.000000
2,0.870388,0.231062
3,NaN,0.108631
4,1.000000,0.796970
5,0.298279,0.447718
...,...,...
939,0.294174,0.534390
940,-0.113961,0.213858
941,0.500000,0.125724


In [13]:
F=Final_ratings.dropna(axis=0, how='any')
print(F)

             1_x       1_y
userId                    
1       1.000000  1.000000
2       0.870388  0.231062
4       1.000000  0.796970
5       0.298279  0.447718
6       0.167319  0.349922
...          ...       ...
939     0.294174  0.534390
940    -0.113961  0.213858
941     0.500000  0.125724
942    -0.866025 -0.185959
943     0.456595  0.068821

[670 rows x 2 columns]


In [79]:
F.sort_values(by='1_x').tail(30)

,1_x,1_y
userId,,
135,1.0,0.502132
132,1.0,-0.166667
736,1.0,0.749520
414,1.0,0.715748
169,1.0,0.686406
700,1.0,0.538734
697,1.0,0.638435
683,1.0,0.400306
680,1.0,0.104322


In [83]:
from sklearn.metrics import ndcg_score

In [91]:
D=[[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]]
C=[[0.1446,0.2914,0.7333,1.0,0.5200,0.6602,0.6943,0.2728,0.6232,-0.3150,0.1042,0.6222,0.1905,0.0082,0.3638,0.7276,0.2309,-0.1028,0.5289,0.0775,0.2784,0.1043,0.4003,0.6384,0.5384,0.6864,0.7157,0.7495,-0.1666,0.5021]]

In [92]:
ndcg_score(C,D, k=10)

0.5135771454969495

In [14]:
# sort the dataframe
Final_ratings.sort_values(by='1_x',ascending=False,inplace=True)
# remove rows with missing values
Final_ratings.dropna(inplace=True)
# convert ratings to binary labels
threshold = 0.8
Final_ratings = Final_ratings >= threshold
# view results
Final_ratings.head(5)

,1_x,1_y
userId,,
678,True,False
649,True,False
1,True,True
134,True,False
750,True,False


In [22]:
# sort the dataframe
C=C.sort_values(ascending=False,inplace=True)
# remove rows with missing values
C.dropna(inplace=True)
# convert ratings to binary labels
threshold = 0.8
Final_ratings = Final_ratings >= threshold
# view results
C.head(5)

ValueError: This Series is a view of some other array, to sort in-place you must create a copy

In [17]:
k = 10
print('Recall@k: {:.2f} for k={}'.format(recall_at_k(Final_ratings,k),k))

Recall@k: 0.09 for k=10


In [23]:
ndcg_score(Final_ratings,Final_ratings, k=10)

0.1582089552238806